In [1]:
import pandas as pd
import numpy as np

from scipy import stats
from scipy.stats import uniform
from sympy import *
from scipy.stats import rv_continuous

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objects as go


pd.options.display.float_format = '{:,.4f}'.format

import warnings
warnings.filterwarnings("ignore")

from typing import Tuple

%matplotlib inline

## Задача 1

Отдел технического контроля проверил n партий однотипных изделий и установил, что число нестандартных изделий в одной партии имеет эмпирическое распределение, приведенное в таблице, в одной строке которой указано количество $x_i$ нестандартных изделий в одной партии, а в другой строке – количество $n_i$ партий, содержащих $x_i$ нестандартных изделий. Требуется при уровне значимости $α = 0,05$ проверить гипотезу о том, что случайная величина $X$ (число нестандартных изделий в одной партии) распределена по закону Пуассона.

| $x_i$ | 0 | 1 | 2 | 3 | 4 | 5 |
| --- | --- | --- | --- | --- | --- | --- |
| $n_i$ | 403 | 370 | 167 | 46 | 12 | 2 |

In [15]:
df = pd.DataFrame({'x_i':np.arange(0, 6), 'n_i':[403, 370, 167, 46, 12, 2]})

df

,x_i,n_i
0,0,403
1,1,370
2,2,167
3,3,46
4,4,12
5,5,2


#### Так как в интервале 5 содержится всего 2 наблюдения, необходимо объединить его с соседним интервалом

In [16]:
test_df = df.copy(deep=True)


for index, value in enumerate(test_df['n_i']): 
    if value < 5:
        test_df.loc[test_df.index==index-1, 'n_i'] = test_df.iloc[index][1] + test_df.iloc[index-1][1]
        test_df.drop([index], inplace=True)

#### находим $\lambda$ по методу моментов

In [17]:
lambda_x = round(np.average(a=test_df['x_i'], weights=test_df['n_i']), 1)

lambda_x

0.9

#### Находим теоретические частоты

In [18]:
test_df['n_t'] = np.around(stats.poisson.pmf(test_df['x_i'], lambda_x), 5) * 1000

In [19]:
test_df.loc[test_df.index==test_df.shape[0] - 1, 'n_t'] = (1 - stats.poisson.cdf(3.9, 0.9)) * 1000

In [20]:
test_df['stats'] = (test_df['n_i'] - test_df['n_t'])**2/test_df['n_t']

In [21]:
test_df

,x_i,n_i,n_t,stats
0,0,403,406.5700,0.0313
1,1,370,365.9100,0.0457
2,2,167,164.6600,0.0333
3,3,46,49.4000,0.2340
4,4,14,13.4587,0.0218


In [22]:
test_df.stats.sum()

0.3660949114430707

In [23]:
1 - stats.chi2.cdf(test_df.stats.sum(), 3)

0.9471538963517317

In [10]:
stats.chisquare(test_df['n_i'], f_exp=test_df['n_t'], ddof=1)

Power_divergenceResult(statistic=0.36599006771231996, pvalue=0.947174969681593)

## Задача 2

Используя критерий Пирсона, при уровне значимости $a = 0,05$ проверить, согласуется ли гипотеза о нормальном распределении генеральной совокупности $X$ с эмпирическим распределением выборки объема $n = 200$.

| x_i | 5 | 7 | 9 | 11 | 13 | 15 | 17 | 19 | 21 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| n_i | 15 | 26 | 25 | 30 | 26 | 21 | 24 | 20 | 13 |

In [27]:
df = pd.DataFrame({'x_i':np.arange(5, 22, 2), 'n_i':[15, 26, 25, 30, 26, 21, 24, 20, 13]})

df['w_i'] = df['n_i'] / df['n_i'].sum()

xk = df['x_i']
pk = df['w_i']
custm_dist = stats.rv_discrete(name='cust_dist', values=(xk, pk))

df

,x_i,n_i,w_i
0,5,15,0.0750
1,7,26,0.1300
2,9,25,0.1250
3,11,30,0.1500
4,13,26,0.1300
5,15,21,0.1050
6,17,24,0.1200
7,19,20,0.1000
8,21,13,0.0650


#### находим $\mu$ и $\sigma$ по методу моментов

In [28]:
mean = custm_dist.mean()
var = custm_dist.var()
std = custm_dist.std()

pd.DataFrame(data=[[mean, var, std]], 
             columns=['mean',' var', 'std'])

,mean,var,std
0,12.6300,22.0431,4.6950


In [31]:
# Умножаем еще и на 2, так как это шаг между вариантами

df['n_t'] = np.around(stats.norm(loc=mean, scale=std).pdf(df['x_i']), 5) * df['n_i'].sum() * 2
df.loc[df.index==df.shape[0] - 1, 'n_t'] = 0 
df.loc[df.index==df.shape[0] - 1, 'n_t'] = (df['n_i'].sum() - df['n_t'].sum())


In [41]:
df

,x_i,n_i,w_i,n_t
0,5,15,0.0750,9.0760
1,7,26,0.1300,16.5600
2,9,25,0.1250,25.2080
3,11,30,0.1500,32.0000
4,13,26,0.1300,33.8840
5,15,21,0.1050,29.9240
6,17,24,0.1200,22.0400
7,19,20,0.1000,13.5400
8,21,13,0.0650,17.7680


In [42]:
df['stats'] = (df['n_i'] - df['n_t'])**2/df['n_t']

In [43]:
df

,x_i,n_i,w_i,n_t,stats
0,5,15,0.0750,9.0760,3.8667
1,7,26,0.1300,16.5600,5.3813
2,9,25,0.1250,25.2080,0.0017
3,11,30,0.1500,32.0000,0.1250
4,13,26,0.1300,33.8840,1.8344
5,15,21,0.1050,29.9240,2.6613
6,17,24,0.1200,22.0400,0.1743
7,19,20,0.1000,13.5400,3.0821
8,21,13,0.0650,17.7680,1.2795


In [108]:
df.stats.sum()

18.406262727994445

In [109]:
1 - stats.chi2.cdf(df.stats.sum(), len(df)-3)

0.005293216104926279

In [110]:
stats.chisquare(df['n_i'], f_exp=df['n_t'], ddof=2)

Power_divergenceResult(statistic=18.406262727994445, pvalue=0.005293216104926286)